In [2]:
pip install dash pandas

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for dash from https://files.pythonhosted.org/packages/b2/10/388c4a697275417a6974033e6ea7235d61e648e6c39d9cc06fcc6a6f71d4/dash-2.15.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 16.2 MB/s eta 0:00:0000:010:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [79]:
# Import libraries
# For interactive web applications
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
# For data manupilation
import pandas as pd
import numpy as np  
# For graphs
import plotly.graph_objs as go  
import plotly.express as px 

In [80]:
# File path for file
file_path = 'Conditionally Admitted Students.xlsx'  
# Read the Excel file
df = pd.read_excel(file_path, sheet_name='Conditional Admits')

In [81]:
# Initialize Dash app
app = dash.Dash(__name__)

# Define layout
app.layout = html.Div([
    # Create a space for the the widgets to pick variables
    html.Div([
        # Dropdown for selecting the X variable
        dcc.Dropdown(
            id='x-variable-selector',
            # Insert the columns in the datasheet as the variables
            options=[{'label': i, 'value': i} for i in df.columns],
            # Default Valye
            value='ETHNICITY',
            # Format the widget
            style={'width': '48%', 'display': 'inline-block'}
        ),
        # Dropdown for selecting the Y variable
        dcc.Dropdown(
            id='y-variable-selector', 
            # Insert the columns in the datasheet as the variables (Categorical)
            options=[{'label': i, 'value': i} for i in df.columns if df[i].dtype == 'O'],  
            # Default Valye
            value='AGE',
            # Format the widget
            style={'width': '48%', 'display': 'inline-block', 'marginLeft': '4%'}  # Style the dropdown, with added left margin
        )
    ]),
    # Placeholder so it can be updated (Display)
    dcc.Graph(id='variable-relationship-graph')
])

In [82]:
# Callback function to update variables based on user
@app.callback(
    # Specifies what to update
    Output('variable-relationship-graph', 'figure'),
    # Specifies the inputs that will be triggers 
    [Input('x-variable-selector', 'value'),
     Input('y-variable-selector', 'value')]
)

# Function that updates graphs
def update_graph(x_var, y_var):
    
    # Generate clustered bar chart
        # Prepare data
            # Group the data inorder to get counts
    grouped_data = df.groupby([x_var, y_var]).size().reset_index(name='Counts')
    
        # Create the chart
            # Y variable is used the color the charts 
    fig = px.bar(grouped_data, x=x_var, y='Counts', color=y_var, barmode='group',
                 title=f'Clustered Bar Chart: {y_var} within {x_var}')
    
    return fig

In [83]:
# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)